## What the important is not the arrival point, it is the journey itself. Enjoy your journey :)

# Including 
* Single-Particle Theta and Real Space Hopping Phases
* Essential First Band Approximation Functions

In [25]:
using NBInclude; using LinearAlgebra
@nbinclude("../Hofstadter Single Particle in Theta Space.ipynb")
@nbinclude("../../First Band Approximation Functions.ipynb");

# Initial Parameters

In [41]:
# Lattice Size
Nx=3; Ny=3

# Magnetic Flux Per Plaquet
p=1; q=Ny

# Total Particle Number
# NON-INTERACTING CASE:
# PN = 1

# INTERACTING CASE
# PN = [0, 1, 2]

# Cut-Off for Sub-Space (Full Space for now...)
# cut_off =  Nx*Ny

# Interaction Potential
U = 2

# Theta Parameter Space
#T_size = 5;

# Single Particle Operator

In [42]:
function Sp_Op(Nx, Ny, p, q, Tx, Ty)

    N=Nx*Ny
    sp_basis = NLevelBasis(N) 
    
    H_T = HSP_T(Nx, Ny, alpha, Tx, Ty, 0)
    
    H = SparseOperator(sp_basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

Sp_Op (generic function with 1 method)

In [49]:
Tx=Ty=0
E_sp, V_sp = eigenstates(dense(Sp_Op(Nx, Ny, p, q, Tx, Ty)))
V_sp

9-element Vector{Ket{NLevelBasis{Int64}, Vector{ComplexF64}}}:
 Ket(dim=9)
  basis: NLevel(N=9)
 -0.11914622089860404 - 0.5071558242595042im
 -0.21023517588349272 - 0.47443535494995515im
 -0.13253848174729294 - 0.39262817925003207im
 -0.03227352354697386 - 0.220407625447626im
 -0.10408041719825618 - 0.1915026151360129im
 -0.03272046930954956 - 0.09108895498488873im
  0.04953365215294943 - 0.29810431958382494im
 -0.21860806220772788 - 0.20489487598470207im
                  0.0 + 0.0im
 Ket(dim=9)
  basis: NLevel(N=9)
  0.05283121635129683 - 0.0915063509461098im
  0.05283121635129684 + 0.09150635094610984im
  0.39433756729740654 + 2.0e-17im
 -0.10566243270259378 + 4.0e-17im
 -0.10566243270259376 - 2.0e-17im
  0.39433756729740654 + 2.0e-17im
 -0.19716878364870327 - 0.34150635094610976im
 -0.19716878364870347 + 0.3415063509461098im
   0.5773502691896255 - 0.0im
 Ket(dim=9)
  basis: NLevel(N=9)
 -0.21344349122547565 + 0.07216514163657863im
  0.07879241070598719 - 0.21603889570766988im
  0.

In [50]:
EE_sp, VV_sp = eigen(dense(Sp_Op(Nx, Ny, p, q, Tx, Ty)).data)
VV_sp

9×9 Matrix{ComplexF64}:
  -0.119146-0.507156im  0.0528312-0.0915064im    …  -0.415746-0.106371im
  -0.210235-0.474435im  0.0528312+0.0915064im       0.0850557+0.361374im
  -0.132538-0.392628im   0.394338+1.62352e-17im       0.33069-0.255004im
 -0.0322735-0.220408im  -0.105662+3.51927e-17im      0.355487+0.107027im
   -0.10408-0.191503im  -0.105662-2.34618e-17im     -0.115753-0.413232im
 -0.0327205-0.091089im   0.394338+1.62352e-17im  …   -0.33069+0.255004im
  0.0495337-0.298104im  -0.197169-0.341506im        0.0602589-0.000655909im
  -0.218608-0.204895im  -0.197169+0.341506im        0.0306975+0.0518578im
        0.0+0.0im         0.57735-0.0im                   0.0+0.0im

# Sub-Space Many-Body

In [185]:
@nbinclude("../Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")
function sub_H(Nx, Ny, p, q, Tx, Ty, cut_off, PN)

    N=Nx*Ny
    alpha = p/q
    
    # Single Particle Operator
    H1 = Sp_Op(Nx, Ny, p, q, Tx, Ty)
    
    # Sub-States Go on to Cut-Off
    sub_states = get_sub_states(H1, cut_off)
    
    sp_basis = NLevelBasis(N) 
    
    # Sub-Space Basis and Projection Operator
    basis_sub, P, Pt = get_projector_op(sub_states, sp_basis)

    # Single-Particle Sub-Space Operator
    H1_sub = get_subspace_op(H1, P, Pt)

    # Many-Body Sub-Space Bosonic Fock States
    states_mb = bosonstates(basis_sub, PN)     

    # Many-Body Sub-Space Basis
    basis_mb = ManyBodyBasis(basis_sub, states_mb)

    basis_cut_mb, basis_cut_sp = get_Bosonic_MB_Basis(cut_off,PN);
    
    # Kinetic Term of Many-Body Sub-Space Operator
    H1_MB = get_mb_op(basis_mb, basis_sub, H1_sub)
    H_kin = SparseOperator(basis_cut_mb)
    H_kin.data = H1_MB.data

    # Interaction Term of Many-Body Sub-Space Operator
    #H_Int = Hubbard_Interaction(P, Pt, basis_cut_mb, cut_off, U)
    
    return H_kin
end

sub_H (generic function with 1 method)

In [186]:
Tx = Ty = 0
cut_off =  Nx*Ny
PN = 1;

In [187]:
E_mb, V_mb = eigenstates(dense( (sub_H(Nx, Ny, p, q, Tx, Ty, cut_off, PN)+sub_H(Nx, Ny, p, q, Tx, Ty, cut_off, PN)')/2 ))
V_mb

9-element Vector{Ket{ManyBodyBasis{Vector{Int64}, NLevelBasis{Int64}, 0xe4e4baeab48fcd49, UInt64}, Vector{ComplexF64}}}:
 Ket(dim=9)
  basis: ManyBody(onebodybasis=NLevel(N=9), states:9)
 -0.02123174865941653 + 0.10564147199067923im
  -0.5354746924325581 + 0.8341289164491571im
 -0.02458258506997005 + 0.07266769064306317im
              3.0e-17 - 1.2e-16im
             -2.0e-17 - 4.0e-17im
             -3.1e-16 + 4.3e-16im
             -1.1e-16 + 1.6e-16im
              7.0e-17 - 5.0e-17im
                  0.0 - 0.0im
 Ket(dim=9)
  basis: ManyBody(onebodybasis=NLevel(N=9), states:9)
  0.22768685118156337 - 0.09547546849024292im
 -0.09486724412515558 - 0.0299386056369294im
   0.9486625031502366 + 0.17084049202965787im
              1.0e-17 + 1.0e-16im
             -9.0e-17 + 2.0e-17im
              2.0e-17 - 1.0e-17im
             -1.7e-16 + 1.7e-16im
                  0.0 + 9.0e-17im
              3.4e-16 - 0.0im
 Ket(dim=9)
  basis: ManyBody(onebodybasis=NLevel(N=9), states:9)
    0.9

In [188]:
EE_mb, VV_mb = eigen(dense((sub_H(Nx, Ny, p, q, Tx, Ty, cut_off, PN)+sub_H(Nx, Ny, p, q, Tx, Ty, cut_off, PN)')/2).data)
VV_mb[:,1:3]

9×3 Matrix{ComplexF64}:
   -0.0212317+0.105641im     …      0.945013+0.185426im
    -0.535475+0.834129im          -0.0776814-0.0395638im
   -0.0245826+0.0726677im           -0.18573-0.174564im
  3.14419e-17-1.1601e-16im      -6.10623e-16-1.80411e-16im
 -1.84043e-17-3.84079e-17im     -1.38778e-16-6.93889e-17im
 -3.11058e-16+4.31512e-16im  …   1.73472e-16+3.1225e-17im
 -1.13258e-16+1.61275e-16im     -2.02963e-16+9.71445e-17im
  6.59195e-17-5.30175e-17im     -6.66134e-16-1.45717e-16im
  4.45222e-18-0.0im             -3.08242e-16-0.0im

In [189]:
for (i,j) in enumerate(4:6)
    println(i,j)
end

14
25
36


In [201]:
function conv_mat(Nx,Ny,q,n1,n2,V::Matrix{ComplexF64})
    v = zeros(Complex{Float64},Nx*Ny,q)
    for (i,j) in enumerate(n1:n2)
        v[:,i]=reshape(V[j].data,Nx*Ny,1)
    end
    return v
end

conv_mat (generic function with 3 methods)

In [203]:
H = sub_H(Nx, Ny, p, q, tx+dx, ty+dy, cut_off, PN)
#w4, v4 = eigen(dense((H+H')/2).data);v4 = v4[:,n1:n2]
w4, v4 = eigenstates(dense((H+H')/2))
v4 = conv_mat(Nx, Ny, q, n1, n2, v4)

9×3 Matrix{ComplexF64}:
    0.160604+0.0im      0.111966+0.0im      0.888115+0.0im
  -0.0915554+0.0im    -0.0180398+0.0im     -0.361824+0.0im
    0.720019+0.0im      0.259064+0.0im     -0.230252+0.0im
 3.04725e-16+0.0im  -7.33982e-17+0.0im  -2.22045e-16+0.0im
 3.35329e-16+0.0im   2.22531e-16+0.0im  -1.00614e-16+0.0im
 3.01929e-16+0.0im  -6.08393e-17+0.0im    1.9984e-15+0.0im
 1.61118e-16+0.0im  -8.71484e-17+0.0im   1.09541e-16+0.0im
 1.11088e-16+0.0im  -8.54527e-17+0.0im           0.0+0.0im
         0.0+0.0im           0.0+0.0im           0.0+0.0im

# Exact Diagonalization

In [26]:
# using DataFrames

# function get_filtered_energies(pn, basis, H_total)
    
#     # Operator Form of Eigenstates
#     E, V = eigenstates(dense(dense((H_total+H_total')/2)))

#     # Energies According to Each Particle Number
#     PN_Energies = Array{Float64}(undef, length(E), 2)
#     for i in 1:length(E)
#         # Particle Number in Each State
#         PN_Energies[i] = round(expect(number(basis), V[i]))
        
#         # Energies in Each State
#         PN_Energies[i,2] = E[i]
#     end
    
#     # Fiter Process
#     Filtered_Energies = DataFrame(PN_Energies, :auto)
#     Filtered_Energies = filter(row -> (row.x1 == pn),  Filtered_Energies)
    
#     return Filtered_Energies
# end

In [27]:
# # Energies of Filtered Particle Number 
# pn = 1
# df = get_filtered_energies(pn, basis_cut_mb, H_total)
# filtered_energies = Matrix(df)[:,2]

In [28]:
# using Plots

# # Plots Bands
# scatter(1:length(filtered_energies), filtered_energies, legend=false, axis=nothing)

In [29]:
# # Eigen-States of Filtered Particle Number 
# function Filtered_Hubbard_States(filtered_energies, states)
#     number_of_states = length(filtered_energies)
#     return states[:,1:number_of_states]
# end

In [30]:
# pn = 1

# function get_eigenstates(basis_cut_mb, H_total, pn)
    
#     E, V = eigen(dense((H_total+H_total')/2).data)

#     # Sorting Again...
#     #V = V[:, sortperm(E)]
    
#     df = get_filtered_energies(pn, basis_cut_mb, H_total)
    
#     filtered_energies = Matrix(df)[:,2]
    
#     #filtered_states = Filtered_Hubbard_States(filtered_energies, V)
#     #filtered_states = dropcol(V, 4)
    
#     return filtered_energies, V
# end

# # Orthogonality 
# # filtered_energies, filtered_states = get_eigenstates(basis_cut_mb, H_total, pn)
# # for i in 1:N
# #     j = 9
# #     a = filtered_states[:,j]
# #     b = filtered_states[:,i]
# #     println(dot(a,b))
# # end
# Tx = Ty = 0
# H_total = get_total_H(Nx, Ny, alpha, sp_basis, Tx, Ty, cut_off, PN, basis_cut_sp, basis_cut_mb)
# get_eigenstates(basis_cut_mb, H_total, pn)[1]

# CHERN CALCULATION SECTION

In [12]:
# function findcol(M, col)                
#     @inbounds @views for c in axes(M, 2)
#        M[:,c] == col && return c       
#     end                                 
#     return nothing                      
# end
# col = [
#     1.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im
# ]
# dropcol(M::AbstractMatrix, j) = M[:, deleteat!(collect(axes(M, 2)), j)];

In [204]:
# Mesh Grid for Theta Space Parameter
T_size = 5;
dx=2*pi/T_size; dy=dx;
Tx=collect(range(start=0, stop=2*pi-dx, step=dx))
Ty=collect(range(start=0, stop=2*pi-dy, step=dy));

In [206]:
n1=1
n2=3
Sum=0
for tx in Tx
    for ty in Ty    
        
        H = sub_H(Nx, Ny, p, q, tx, ty, cut_off, PN)
        #w1, v1 = eigen(dense((H+H')/2).data);v1 = v1[:,n1:n2]
        w1, v1 = eigenstates(dense((H+H')/2))
        v1 = conv_mat(Nx, Ny, q, n1, n2, v1)
        
        H = sub_H(Nx, Ny, p, q, tx+dx, ty, cut_off, PN)
        #w2, v2 = eigen(dense((H+H')/2).data);v2 = v2[:,n1:n2]
        w2, v2 = eigenstates(dense((H+H')/2))
        v2 = conv_mat(Nx, Ny, q, n1, n2, v2)
        
        H = sub_H(Nx, Ny, p, q, tx, ty+dy, cut_off, PN)
        #w3, v3 = eigen(dense((H+H')/2).data);v3 = v3[:,n1:n2]
        w3, v3 = eigenstates(dense((H+H')/2))
        v3 = conv_mat(Nx, Ny, q, n1, n2, v3)

        H = sub_H(Nx, Ny, p, q, tx+dx, ty+dy, cut_off, PN)
        #w4, v4 = eigen(dense((H+H')/2).data);v4 = v4[:,n1:n2]
        w4, v4 = eigenstates(dense((H+H')/2))
        v4 = conv_mat(Nx, Ny, q, n1, n2, v4)
        
        U1 = det(v1'*v2)/abs(det(v1'*v2))
        U2 = det(v2'*v4)/abs(det(v2'*v4))
        U3 = det(v3'*v4)/abs(det(v3'*v4))
        U4 = det(v1'*v3)/abs(det(v1'*v3))

        Sum+=log(U1*U2*1/U3*1/U4)
    end
end
Sum/2*1im*pi

0.0 + 0.0im

In [84]:
function Chern_Nums(Nx, Ny, p, q, Tx, Ty, cut_off, PN)
    
    Sum=0
    
    for tx in Tx
        for ty in Ty
            
            H = sub_H(Nx, Ny, p, q, tx, ty, cut_off, PN)
            
            w1, v1 = eigen(dense((H+H')/2).data)
            
            # v1 = v1[:, sortperm(w1)]
            
            # if isapprox(v1[1,1], 0+0im, atol=0.001) == false
            #     v1 = v1 ./ v1[1,1]
            #     v1 = v1/norm(v1)
            # end

            # col_idx = findcol(v1, col)
            # v1 = dropcol(v1, col_idx)
            
            v1 = v1[:,n1:n2]
            
            #------------------------------------
            
            H = sub_H(Nx, Ny, p, q, tx+dx, ty, cut_off, PN)
            
            w2, v2 = eigen(dense((H+H')/2).data)
            
            # v2 = v2[:, sortperm(w2)]

            # if isapprox(v2[1,1], 0+0im, atol=0.001) == false
            #     v2 = v2 ./ v2[1,1]
            #     v2 = v2 / norm(v2)
            # end

            # col_idx = findcol(v2, col)
            # v2 = dropcol(v2, col_idx)
            
            v2 = v2[:,n1:n2]
            
            #------------------------------------
            
            H = sub_H(Nx, Ny, p, q, tx, ty+dy, cut_off, PN)
            
            w3, v3 = eigen(dense((H+H')/2).data)
            
            # v3 = v3[:, sortperm(w3)]

            # if isapprox(v3[1,1], 0+0im, atol=0.001) == false
            #     v3 = v3 ./ v3[1,1]
            #     v3 = v3/norm(v3)
            # end

            # col_idx = findcol(v3, col)
            # v3 = dropcol(v3, col_idx)
            
            v3 = v3[:,n1:n2]
            
            #------------------------------------
            
            H = sub_H(Nx, Ny, p, q, tx+dx, ty+dy, cut_off, PN)
            
            # w4, v4 = eigen(dense((H+H')/2).data)
            
            # v4 = v4[:, sortperm(w4)]

            # if isapprox(v4[1,1], 0+0im, atol=0.001) == false
            #     v4 = v4 ./ v4[1,1]
            #     v4 = v4/norm(v4)
            # end

            # col_idx = findcol(v4, col)
            # v4 = dropcol(v4, col_idx)
            
            v4 = v4[:,n1:n2]
            
            #----------LINK VARIABLES------------
            
            U1=det(adjoint(v1)*v2)
            U1=U1/abs(U1)
            
            U2=det(adjoint(v2)*v4)
            U2=U2/abs(U2)
            
            U3=det(adjoint(v3)*v4)
            U3=U3/abs(U3)
            
            U4=det(adjoint(v1)*v3)
            U4=U4/abs(U4)
            
            #----------BERRY CURVATURE-----------
            
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
            
        end
    end
    
    C = 1/(2*pi*1im)*Sum
    
    return C
end

Chern_Nums (generic function with 2 methods)

In [85]:
Chern_Nums(Tx, Ty, 1, 3)

LoadError: MethodError: no method matching exp(::Vector{ComplexF64})
[0mClosest candidates are:
[0m  exp([91m::Union{Float16, Float32, Float64}[39m) at special/exp.jl:326
[0m  exp([91m::StridedMatrix{var"#s885"} where var"#s885"<:Union{Float32, Float64, ComplexF32, ComplexF64}[39m) at C:\Users\Can\AppData\Local\Programs\julia-1.8.1\share\julia\stdlib\v1.8\LinearAlgebra\src\dense.jl:569
[0m  exp([91m::StridedMatrix{var"#s885"} where var"#s885"<:Union{Integer, Complex{<:Integer}}[39m) at C:\Users\Can\AppData\Local\Programs\julia-1.8.1\share\julia\stdlib\v1.8\LinearAlgebra\src\dense.jl:570
[0m  ...

# Check with "Hofstadter MB Chern" file

In [69]:
n1 = 1
n2 = 9
tx = ty = 0

0

In [71]:
H = get_total_H(Nx, Ny, alpha, tx, ty, cut_off, PN)

w1, v1 = eigen(dense((H+H')/2).data)

v1 = v1[:, sortperm(w1)]

if isapprox(v1[1,1], 0+0im, atol=0.001) == false
    v1 = v1 ./ v1[1,1]
    v1 = v1/norm(v1)
end

# col_idx = findcol(v1, col)
# v1 = dropcol(v1, col_idx)

v1 = v1[:,n1:n2]

#------------------------------------

H = get_total_H(Nx, Ny, alpha, tx+dx, ty, cut_off, PN)

w2, v2 = eigen(dense((H+H')/2).data)

v2 = v2[:, sortperm(w2)]

if isapprox(v2[1,1], 0+0im, atol=0.001) == false
    v2 = v2 ./ v2[1,1]
    v2 = v2 / norm(v2)
end

# col_idx = findcol(v2, col)
# v2 = dropcol(v2, col_idx)

v2 = v2[:,n1:n2]

#------------------------------------

H = get_total_H(Nx, Ny, alpha, tx, ty+dy, cut_off, PN)

w3, v3 = eigen(dense((H+H')/2).data)

v3 = v3[:, sortperm(w3)]

if isapprox(v3[1,1], 0+0im, atol=0.001) == false
    v3 = v3 ./ v3[1,1]
    v3 = v3/norm(v3)
end

# col_idx = findcol(v3, col)
# v3 = dropcol(v3, col_idx)

v3 = v3[:,n1:n2]

#------------------------------------

H = get_total_H(Nx, Ny, alpha, tx+dx, ty+dy, cut_off, PN)

w4, v4 = eigen(dense((H+H')/2).data)

v4 = v4[:, sortperm(w4)]

if isapprox(v4[1,1], 0+0im, atol=0.001) == false
    v4 = v4 ./ v4[1,1]
    v4 = v4/norm(v4)
end

# col_idx = findcol(v4, col)
# v4 = dropcol(v4, col_idx)

v4 = v4[:,n1:n2]

#----------LINK VARIABLES------------

U1=det(adjoint(v1)*v2)
U1=U1/abs(U1)

U2=det(adjoint(v2)*v4)
U2=U2/abs(U2)

U3=det(adjoint(v3)*v4)
U3=U3/abs(U3)

U4=det(adjoint(v1)*v3)
U4=U4/abs(U4)

#----------BERRY CURVATURE-----------

F=log(U1*U2*1/U3*1/U4)

8.52741319615994e-32 - 4.129748950277714e-16im

In [73]:
eigen(v1)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
9-element Vector{ComplexF64}:
 -0.33171144944039976 + 0.0328424332420734im
 -0.32944251696367644 - 0.050781287180897416im
 -0.31160750339306503 + 0.11837176580693569im
  -0.2689834502969823 + 0.19687309256839067im
 -0.16947438557723316 - 0.2870357882639909im
  0.09547931048700889 + 0.31936626681607505im
   0.2237484943774718 - 0.24707837294050172im
   0.3326247298839208 + 0.02172326358446125im
  0.33331581880936106 - 0.003417022469445724im
vectors:
9×9 Matrix{ComplexF64}:
    0.0773191+0.511891im     …   3.08526e-16+1.62928e-19im
     0.129845-0.56231im          1.64921e-16-1.29095e-16im
     0.631615+0.0im             -1.48585e-16-1.38088e-16im
  -2.3032e-16-4.75835e-16im         0.751813+0.0im
 -2.09388e-17-1.91545e-16im        -0.104663-0.021228im
 -1.77055e-16+3.33217e-16im  …      0.243715+0.603304im
 -4.12767e-16+3.11066e-16im       3.0247e-15-3.14179e-15im
 -2.24324e-16-1.04232e-16im      4.97207e-15-2

In [63]:
PN = 1
tx = 1
ty = 1
#---
H = get_total_H(Nx, Ny, alpha, Tx[tx], Ty[ty], cut_off, PN)     
w1, v1 = eigen(dense((H+H')/2).data)
v1 = v1[:, sortperm(w1)]
# drop_idx = findcol(v1, col)
# v1 = dropcol(v1, drop_idx)
if isapprox(v1[1,1], 0+0im, atol=0.001) == false
    v1 = v1 ./ v1[1,1]
    v1 = v1/norm(v1)
end
v1 = v1[:,n1:n2]
#---
H = get_total_H(Nx, Ny, alpha, Tx[tx]+dx, Ty[ty], cut_off, PN)     
w2, v2 = eigen(dense((H+H')/2).data)
v2 = v2[:, sortperm(w2)]
# drop_idx = findcol(v2, col)
# v2 = dropcol(v2, drop_idx)
v2 = v2[:,n1:n2]
#---
H = get_total_H(Nx, Ny, alpha, Tx[tx], Ty[ty]+dy, cut_off, PN)     
w3, v3 = eigen(dense((H+H')/2).data)
v3 = v3[:, sortperm(w3)]
# drop_idx = findcol(v3, col)
# v3 = dropcol(v3, drop_idx)
v3 = v3[:,n1:n2]
#---
H = get_total_H(Nx, Ny, alpha, Tx[tx]+dx, Ty[ty]+dy, cut_off, PN)     
w4, v4 = eigen(dense((H+H')/2).data)
v4 = v4[:, sortperm(w4)]
# drop_idx = findcol(v4, col)
# v4 = dropcol(v4, drop_idx)
v4 = v4[:,n1:n2]
#---
U1=det(adjoint(v1)*v2)
U1=U1/abs(U1)
U2=det(adjoint(v2)*v4)
U2=U2/abs(U2)
U3=det(adjoint(v3)*v4)
U3=U3/abs(U3)
U4=det(adjoint(v1)*v3)
U4=U4/abs(U4)

0.09998746114914639 - 0.9949886972287413im

In [64]:
w4

9-element Vector{Float64}:
 -2.546639394327162
 -2.5466393943271606
 -2.546639394327156
  0.20750036611082465
  0.20750036611082553
  0.20750036611082567
  2.339139028216335
  2.339139028216336
  2.3391390282163362

In [65]:
v1[:,2:2]

9×1 Matrix{ComplexF64}:
    0.22768685118156337 - 0.09547546849024292im
   -0.09486724412515558 - 0.029938605636929395im
     0.9486625031502366 + 0.17084049202965787im
 1.3877787807814457e-17 + 9.71445146547012e-17im
 -9.020562075079397e-17 + 1.5612511283791264e-17im
   2.42861286636753e-17 - 6.938893903907228e-18im
 -1.734723475976807e-16 + 1.734723475976807e-16im
                    0.0 + 9.367506770274758e-17im
  3.354703270462921e-16 - 0.0im

In [53]:
A = [0.04206576838675055 - 0.4844396754976562im
  -0.11575260100874928 - 0.5048774472308879im
 -0.019490509012952906 - 0.41988073170016393im
   0.08258610549150286 - 0.15124676786746452im
  -0.13712915132022854 - 0.206737076416468im
   0.02043777173323122 - 0.1578183693955im
    0.1675828210222271 - 0.2475088598632606im
  -0.20168809511772048 - 0.2682933538161717im
                   0.0 + 0.0im]

9-element Vector{ComplexF64}:
   0.04206576838675055 - 0.4844396754976562im
  -0.11575260100874928 - 0.5048774472308879im
 -0.019490509012952906 - 0.41988073170016393im
   0.08258610549150286 - 0.15124676786746452im
  -0.13712915132022854 - 0.206737076416468im
   0.02043777173323122 - 0.1578183693955im
    0.1675828210222271 - 0.2475088598632606im
  -0.20168809511772048 - 0.2682933538161717im
                   0.0 + 0.0im

In [58]:
v1[:,4:4] ./ A

9×1 Matrix{ComplexF64}:
 -1.7980148799796905e-16 + 2.6448446692249435e-16im
  3.1046229679841827e-16 + 8.476994294114351e-16im
 -2.5323280801083318e-17 + 1.227685310490113e-16im
      -1.090534299951408 - 0.4449238617579872im
     0.36100073604058036 + 0.09170231699197172im
     -2.7721760625514626 - 5.462498847891495im
   8.007269689386028e-17 + 1.7066629704076275e-16im
   8.099994278116515e-17 - 7.990563131415732e-16im
                     NaN + NaN*im

In [49]:
eigen(v1)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
9-element Vector{ComplexF64}:
  -0.9957553612966031 + 0.09203945050397394im
 -0.42003967852213075 - 0.9075057401840629im
  -0.2605125273634965 + 0.9654704672265649im
 -0.18697876421055862 + 0.9823639558403457im
 -0.16395678044357687 - 0.9864675231078723im
  0.09948476278099364 - 0.9950390856516179im
  0.34409654116713523 - 0.9389342737150541im
   0.5944419686309548 + 0.8041385116571351im
   0.9444051792358245 - 0.32878390689410203im
vectors:
9×9 Matrix{ComplexF64}:
    -0.503198-0.224812im     …   1.60054e-16-8.15074e-17im
    -0.494572+0.291905im         9.05782e-17+7.01495e-17im
     0.605344+0.0im             -1.78743e-16-8.81097e-18im
  4.41479e-16+3.9254e-17im      -2.13933e-16+1.97689e-16im
 -1.69026e-17+7.64161e-17im      2.57639e-16-5.272e-16im
  8.06173e-17+2.73893e-17im  …   1.03897e-16+2.52044e-16im
  1.17364e-16+1.10632e-16im        -0.596552+0.306689im
  3.10486e-16+2.09361e-16im         0.716812